In [2]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer

import time

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

Using TensorFlow backend.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
mixer.init()
sound = mixer.Sound('./01/alarm.wav')

face = cv2.CascadeClassifier('01/haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('01/haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('01/haarcascade_righteye_2splits.xml')

lbl=['State : Drowsy', 'State : Awake']
model = load_model('cnnCat2_mine.h5')
path = os.getcwd()
font = cv2.FONT_HERSHEY_PLAIN
cap = cv2.VideoCapture(0)
count=0
score=0
thicc=2
danger = 15
rpred=[99]
lpred=[99]

start = True

while(start):
    # ------------------그래프 코드------------------
    
    fig = plt.figure()
    
    x1 = np.linspace(0.0, 5.0)
    x2 = np.linspace(0.0, 2.0)

    y1 = np.cos(2 * np.pi * x1) * np.exp(-x1)
    y2 = np.cos(2 * np.pi * x2)
    line1, = plt.plot(x1, y1, 'ko-')
    

    for i in range(1000):
        img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
        img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        line1.set_ydata( np.cos(2 * np.pi * (x1+i*3.14/2) ) * np.exp(-x1) )
        fig.canvas.draw()

        cv2.imshow("plot", img)
        
        
    # ------------------여기부터는 다시 졸음운전 코드------------------
        ret, frame = cap.read()
        height, width = frame.shape[:2] 
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face.detectMultiScale(gray, minNeighbors=5, scaleFactor=1.1, minSize=(25,25))
        left_eye = leye.detectMultiScale(gray)
        right_eye =  reye.detectMultiScale(gray)

        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x,y) , (x+w,y+h) , (255,100,100) , 2 )

    # ------------------오른쪽 눈------------------
        for (x,y,w,h) in right_eye:
            r_eye = frame[y:y+h, x:x+w]
            count = count+1
            r_eye = cv2.cvtColor(r_eye, cv2.COLOR_BGR2GRAY)
            r_eye = cv2.resize(r_eye, (24,24))
            r_eye= r_eye/255
            r_eye=  r_eye.reshape(24,24,-1)
            r_eye = np.expand_dims(r_eye, axis=0)

            rpred = model.predict_classes(r_eye)
            if(rpred[0]==1):
                lbl='Eyes Opened' 
            if(rpred[0]==0):
                lbl='Eyes Closed'
            break

    # ------------------왼쪽 눈------------------
        for (x,y,w,h) in left_eye:
            l_eye = frame[y:y+h, x:x+w]
            count = count+1
            l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2GRAY)  
            l_eye = cv2.resize(l_eye, (24,24))
            l_eye= l_eye/255
            l_eye = l_eye.reshape(24,24,-1)
            l_eye = np.expand_dims(l_eye, axis=0)

            lpred = model.predict_classes(l_eye) # label: 0은 눈 감음, 1은 눈 뜸으로 인코딩되어 있음
            if(lpred[0]==1):
                lbl='Eyes Opened'   
            if(lpred[0]==0):
                lbl='Eyes Closed'
            break

    # ------------------하단에 눈 떴음여부 & 카운트(점수) 표시 텍스트------------------        
        if(rpred[0]==0 and lpred[0]==0):
            score = score+1
            cv2.putText(frame,"State: Eyes Closed", (10, 50), font, 1.5, (255,255,255), 2, cv2.LINE_AA)
        else:
            score = score-2
            cv2.putText(frame, "State: Eyes Opened", (10, 50), font, 1.5, (255,255,255), 2, cv2.LINE_AA)

        if(score < 0):
            score = 0   
        cv2.putText(frame, 'Count: ' + str(score), (10, 100), font, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
        if(score < danger and score > 7):
            cv2.putText(frame, "You're Drowsy", (20, height//2), font, 5, (255,255,255), 7, cv2.LINE_AA)
            cv2.putText(frame, 'Recommandation : Get some water', (20, height//2+100), font, 2, (255,255,255), 2, cv2.LINE_AA)

    # ------------------점수에 따른 경고창 디스플레이 조건------------------                 
        if(score > danger):
            cv2.imwrite(os.path.join(path,'image.jpg'), frame)
            try:
                sound.play()
            except:  # isplaying = False
                pass
            if(thicc < 20):
                thicc = thicc+500
            else:
                thicc = thicc-500
                if(thicc < 2):
                    thicc = 2
            cv2.rectangle(frame, (0,0), (width,height), (0,0,255), thicc)
            cv2.putText(frame, 'Danger', (165, height//2), font, 5, (255,255,255), 7, cv2.LINE_AA)
            cv2.putText(frame, 'Awake to be Alive', (160, height//2+100), font, 2, (255,255,255), 3, cv2.LINE_AA)

        
        


               
        cv2.imshow('frame', frame)    
        if cv2.waitKey(1) & 0xFF == ord('q'): # 'q' : 종료
            start = False
            break
        
cap.release()
cv2.destroyAllWindows()

C:\Users\wer29\Anaconda3\envs\keras\lib\site-packages\ipykernel_launcher.py:124: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
